<a href="https://colab.research.google.com/github/Julianfried/Diplomatura-en-IA-Instituto-Humai/blob/main/NLP/Aplicaciones_de_BERT_ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/7_BERT/ejercicios/ejercicios_solucion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios Clase 7

In [1]:
# @markdown Setup
# Fist install the library and download the models from github

!pip install transformers

# import the necessary

import torch
from transformers import BertForMaskedLM, BertTokenizer

# create the tokenizer and the model

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
e = model.eval()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.3 MB/s eta 0:00:00


# Ejercicio 1

A partir de la versión de Beto descargada en las celdas de códico anteriores, implementar un pequeño pipeline que prediga la siguiente palabra.

El pipeline debera:
1. Tomar el texto inicial y convertirlo a índices enteros
2. Agregar al final del texto inicial el token correspondiente a máscara `"[MASK]"`
3. Alimentar al modelo con el texto con máscara
4. Reemplazar `"[MASK]"` con la predicción hecha por el modelo
5. Repetir el proceso unas 6 o 7 veces

In [2]:
text = "[CLS] Era otro día de verano,"
mask_tkn = tokenizer.convert_tokens_to_ids("[MASK]")
sep_tkn = tokenizer.convert_tokens_to_ids("[SEP]")


tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
for i in range(20):
    indexed_tokens.append(mask_tkn)
    tokens_tensor = torch.tensor([indexed_tokens])
    predictions = model(tokens_tensor)[0]
    idxs = torch.argsort(predictions[0,-1], descending=True)
    new_tkn = int(idxs[0])
    indexed_tokens[-1] = new_tkn
    if new_tkn == sep_tkn:
      break

print(' '.join(tokenizer.convert_ids_to_tokens(indexed_tokens)))



[CLS] [UNK] otro día de verano , [UNK] otro dia de verano , [UNK] otro dia de verano , [UNK] otro dia de verano , [UNK] otro


# Ejercicio 2

Implemente un pequeño pipepile que genere un beam search. No hace falta que su implemntación no sea óptima en términos de eficiencia temporal.

Para esto debera crear:

Función `children(model, beam, indeces)`:
  * Recibe el modelo `model` a partir del cual generar predicciones.
  * Recibe el tamaño del haz `beam`
  * Recibe un conjunto de índices `indeces` correspondiente la palabra original

La función `children` deberá:
  1. Agregar al final del texto inicial el token correspondiente a máscara `"[MASK]"`
  3. Alimentar al modelo con el texto con máscara
  5. A la salida del modelo BERT tenemos una serie de números que no son probabilidades. Para convertirla a probabilidades debera usar `torch.softmax`
  3. Ordenar las predicciones de la más probable a la menos probable
      * revise la documentación de la función `torch.sort` y comparela con `torch.argsort`
  4. Tomar las primeras predicciones usando `beam`. Si `beam = 5` tomara las 5 primeras predicciones.
  4. Debera guardar como puntaje el logaritmo de las probabilides de las primeras predicciones.
  5. La función debe devolver un `zip` del indice predicho y su puntaje.

Función `new_top(model, beam, top, scores)`:
  * Recibe el modelo `model` a partir del cual generar predicciones.
  * Recibe el tamaño del haz `beam`
  * Recibe un conjunto de las mejores predicciones `top`
  * Recibe el puntaje `scores` de cada una de las predicciones en `top`

La función `new_top` deberá:
  1. Iterar sobre cada elemento de `top` y `scores` usando un zip.
  1. Iterar sobre cada elemento devuelto de la función `children` usando como entrada el puntaje del iterador anterior
  2. A partir de la prediccion y de el puntaje de la función `children` actualizar los índices de `top` y el `score`
  3. Guardar todos los indices acutalizados con su puntaje. Es conveniente agruparlos en una misma estructura para luego ordenalos de mayor a menos
    * debería tener `beam * beam` candidatos
  4. Ordenar los candidatos de mayor a menor y conservar los primeros. Si `beam = 5` tomara las 5 primeras predicciones.
  5. Devuelva los nuevos top y scores

Con estas funciones, solo deberá llamar a la la función `children` una única vez para genrar 10 candidatos y luego hacer iteracioens con la función `new_top` para implementar un beam search

> Recuerde mostrar algunos de los resultado obtenidos

Este proceso no da muy buenos resultados. ¿Por que lo cree?


In [3]:
text = "[CLS] Ese día, desde mi ventana"
beam = 10

def children(model, beam, indeces):
    mask_tkn = tokenizer.convert_tokens_to_ids("[MASK]")
    indeces.append(mask_tkn)
    tokens_tensor = torch.tensor([indeces])
    predictions = model(tokens_tensor)[0]
    new_score, idxs = torch.sort(predictions[0,-1], descending=True)
    new_score = torch.softmax(new_score,0)
    new_score, idxs = new_score[:beam], idxs[:beam]
    new_score = torch.log(new_score)
    return zip(new_score, idxs)

def new_top(model, beam, top, scores):
    sep_tkn = tokenizer.convert_tokens_to_ids("[SEP]")
    eos_tkn = tokenizer.convert_tokens_to_ids("[EOS]")
    stop_tkn = tokenizer.convert_tokens_to_ids(".")
    candidates = []
    for value,indeces in zip(scores, top):
        if value == float('inf'):
            candidates.append((value, indeces))
            continue
        for new_val, new_idx in children(model, beam, indeces):
            new_indeces = indeces.copy()
            new_indeces[-1] = int(new_idx)
            if new_idx == sep_tkn or new_idx == eos_tkn:
                new_val = float('inf')
            if new_idx == stop_tkn:
                new_indeces.append(sep_tkn)
            #"""
            #for i in range((len(new_indeces) +1 )//2 ):
            for i in range(4):
                penalty = 0
                if new_indeces[-1] == new_indeces[- 2 - i]:
                    penalty = -1000
                    for j in range(1,i + 1):
                        if new_indeces[-1 - j] != new_indeces[- 2 - i - j]:
                            penalty = 0
                            break
                    value += penalty
            #"""
            candidates.append((value + float(new_val), new_indeces))
    candidates = sorted(candidates, key = lambda x: x[0], reverse=True)
    candidates = candidates[:beam * beam]
    return zip(*candidates)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
input = [indexed_tokens.copy()]
scores, top = new_top(model, beam, input,[0])
for i in range(5):
    scores, top = new_top(model, beam, top, scores)
    #scores, top = scores[:beam], top[:beam]
scores, top = scores[:20], top[:20]
for value,indeces in zip(scores, top):
    print()
    print(' '.join(tokenizer.convert_ids_to_tokens(indeces)), "//",value)


[CLS] [UNK] día , desde mi ventana , o desde mi ventana , // -8.897114723920822

[CLS] [UNK] día , desde mi ventana , o desde su ventana , // -9.950661174952984

[CLS] [UNK] día , desde mi ventana ( [UNK] ) , desde mi // -10.028698988258839

[CLS] [UNK] día , desde mi ventana , o , desde mi ventana // -10.229723031632602

[CLS] [UNK] día , desde mi ventana , a mi ventana , desde // -10.350532457232475

[CLS] [UNK] día , desde mi ventana , a mi ventana , [UNK] // -10.606883808970451

[CLS] [UNK] día , desde mi ventana ( [UNK] ) ( 1 ) // -10.795615520328283

[CLS] [UNK] día , desde mi ventana ( o ) ventana ( o // -11.050129771232605

[CLS] [UNK] día , desde mi ventana , o desde tu ventana , // -11.059305399656296

[CLS] [UNK] día , desde mi ventana ( [UNK] ) ( 2 ) // -11.18849747441709

[CLS] [UNK] día , desde mi ventana ( [UNK] ) ( 0 ) // -11.229050752706826

[CLS] [UNK] día , desde mi ventana ( [UNK] ) ( 3 ) // -11.257976830936968

[CLS] [UNK] día , desde mi ventana , te deseo y te de

# Ejercicio 3

Tomando de referencia el siguiente código busque un modelo en español de huggingface preentrenado y pruebe la tarea en la que fue preentrenado.

El código a continuación es PoS (Part of Speech) para tipo de palabra, pero usted puede elegir otra tarea al a que se le ha aplicado fine tuning para un modelo de BERT.



In [4]:
from transformers import pipeline

nlp_ner = pipeline(
    "ner",
    model="mrm8488/RuPERTa-base-finetuned-pos",
    tokenizer="mrm8488/RuPERTa-base-finetuned-pos")

text = 'tras el temporal, el joven encontro un empleo temporal'
text = 'Hacia el este fue donde este sujeto se dirigio y se topo con este'


outputs = nlp_ner(text)

for i in outputs:
  print(i['word'], i['entity'])

Some weights of the model checkpoint at mrm8488/RuPERTa-base-finetuned-pos were not used when initializing RobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Hacia VERB
Ġel DET
Ġeste NOUN
Ġfue AUX
Ġdonde PRON
Ġeste DET
Ġsujeto NOUN
Ġse PRON
Ġdiri VERB
gio VERB
Ġy CCONJ
Ġse PRON
Ġtopo VERB
Ġcon ADP
Ġeste PRON


In [5]:
from transformers import pipeline

nlp_qa = pipeline(
    "question-answering",
    model="inigopm/beto-base-spanish-squades2",
    tokenizer="inigopm/beto-base-spanish-squades2")

example = {'context' : 'Para preparar un mate cebado, se coloca la yerba en un recipiente llamado mate o calabaza, hasta las tres cuartas partes del mismo. Luego se tapa con la mano, se coloca boca abajo y se lo agita (esto hace que las partículas más finas queden en la parte superior, y no obstruyan la bombilla). Se lo coloca nuevamente boca arriba y se le agrega un poco de agua tibia o fría cerca del borde. Se deja reposar algunos segundos (hasta que se absorba el agua) y se termina de llenar con agua caliente, hasta aproximadamente 7 u 8 mm del borde, cuidando de que no se moje la yerba de la superficie (el agua caliente debe estar a una temperatura cercana a 80 °C, antes del punto de ebullición). Luego de uno o dos minutos se ensilla, es decir, se coloca la bombilla tapándole la boca con el dedo pulgar y presionando firmemente hasta el fondo. Siempre tiene que quedar más yerba sobre el lado opuesto de la bombilla. Se debe tener cuidado de no remover demasiado la yerba, porque podría taparse la bombilla. La bombilla se debe inclinar en sentido contrario a donde quedó la yerba seca, es decir, para el lado del que va a tomar el mate. ',
 'question' :'¿Dónde se coloca la yerba para preparar un mate cebado?'}

example = {'context' : 'Para preparar un mate cebado, se coloca la yerba en un recipiente llamado mate o calabaza, hasta las tres cuartas partes del mismo. Luego se tapa con la mano, se coloca boca abajo y se lo agita (esto hace que las partículas más finas queden en la parte superior, y no obstruyan la bombilla). Se lo coloca nuevamente boca arriba y se le agrega un poco de agua tibia o fría cerca del borde. Se deja reposar algunos segundos (hasta que se absorba el agua) y se termina de llenar con agua caliente, hasta aproximadamente 7 u 8 mm del borde, cuidando de que no se moje la yerba de la superficie (el agua caliente debe estar a una temperatura cercana a 80 °C, antes del punto de ebullición). Luego de uno o dos minutos se ensilla, es decir, se coloca la bombilla tapándole la boca con el dedo pulgar y presionando firmemente hasta el fondo. Siempre tiene que quedar más yerba sobre el lado opuesto de la bombilla. Se debe tener cuidado de no remover demasiado la yerba, porque podría taparse la bombilla. La bombilla se debe inclinar en sentido contrario a donde quedó la yerba seca, es decir, para el lado del que va a tomar el mate. ',
 'question' :'¿Hasta dónde se llena el mate para preparar un mate cebado?'}


outputs = nlp_qa(example)

outputs


{'score': 0.09096234291791916,
 'start': 101,
 'end': 120,
 'answer': 'tres cuartas partes'}